# Analyzing Mobile App Data

#### For this project, I am working as a data analyst for a company that builds free Android and iOS mobile apps. Profit is entirely based on in-app ads which means we rely on high user count

#### The goal of this project is to help app developers understand what types of apps are more likely to attract more users

# Import data

In [1]:
apple_file = open('AppleStore.csv', encoding="utf8")
google_file = open('googleplaystore.csv', encoding="utf8")
from csv import reader
raw_apple_data = list(reader(apple_file))
raw_google_data = list(reader(google_file))
ios_header = raw_apple_data[0]
ios = raw_apple_data[1:]
android_header = raw_google_data[0]
android = raw_google_data[1:]


In [2]:
"""build function to display specified rows in dataset"""

def explore_data(dataset, start, end, rows_and_columns=False):
    dataset_slice = dataset[start:end]    
    for row in dataset_slice:
        print(row)
        print('\n') # adds a new (empty) line after each row

    if rows_and_columns:
        print('Number of rows:', len(dataset))
        print('Number of columns:', len(dataset[0]))
        

In [3]:
"""
print header of apple data
a dataset containing about 7,000 apps from 2017
"""

explore_data(raw_apple_data, 0, 1)


['id', 'track_name', 'size_bytes', 'currency', 'price', 'rating_count_tot', 'rating_count_ver', 'user_rating', 'user_rating_ver', 'ver', 'cont_rating', 'prime_genre', 'sup_devices.num', 'ipadSc_urls.num', 'lang.num', 'vpp_lic']




In [4]:
explore_data(raw_apple_data, 1, 4)


['284882215', 'Facebook', '389879808', 'USD', '0.0', '2974676', '212', '3.5', '3.5', '95.0', '4+', 'Social Networking', '37', '1', '29', '1']


['389801252', 'Instagram', '113954816', 'USD', '0.0', '2161558', '1289', '4.5', '4.0', '10.23', '12+', 'Photo & Video', '37', '0', '29', '1']


['529479190', 'Clash of Clans', '116476928', 'USD', '0.0', '2130805', '579', '4.5', '4.5', '9.24.12', '9+', 'Games', '38', '5', '18', '1']




In [5]:
"""
print header of google data
a dataset containing about 10,000 apps from 2018
"""

explore_data(raw_google_data, 0, 1)


['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type', 'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver', 'Android Ver']




In [6]:
explore_data(raw_google_data, 1, 4)


['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up']


['Coloring book moana', 'ART_AND_DESIGN', '3.9', '967', '14M', '500,000+', 'Free', '0', 'Everyone', 'Art & Design;Pretend Play', 'January 15, 2018', '2.0.0', '4.0.3 and up']


['U Launcher Lite – FREE Live Cool Themes, Hide Apps', 'ART_AND_DESIGN', '4.7', '87510', '8.7M', '5,000,000+', 'Free', '0', 'Everyone', 'Art & Design', 'August 1, 2018', '1.2.4', '4.0.3 and up']




# Clean data


The readme file attached to the datafiles indicates that row 10472 has an error in it


In [7]:
print(android_header)
print('\n')
print(android[10472])


['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type', 'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver', 'Android Ver']


['Life Made WI-Fi Touchscreen Photo Frame', '1.9', '19', '3.0M', '1,000+', 'Free', '0', 'Everyone', '', 'February 11, 2018', '1.0.19', '4.0 and up']


The category is listed as 1.9, and the rating is listed as 19... on a scale of 0-5. Something has gone wrong. Let's just remove it

In [8]:
del(android[10472])
print(android[10472])


['osmino Wi-Fi: free WiFi', 'TOOLS', '4.2', '134203', '4.1M', '10,000,000+', 'Free', '0', 'Everyone', 'Tools', 'August 7, 2018', '6.06.14', '4.4 and up']


# Remove duplicate apps

#### Google/Android

In [9]:
unique_apps_android = []
for row in android:
    name = row[0]
    if name not in unique_apps_android:
        unique_apps_android.append(name)
print('length of ios df:', len(android))
print('length of unique_apps_android:', len(unique_apps_android))
      

length of ios df: 10840
length of unique_apps_android: 9659


In [10]:
for row in android:
    if row[0] == 'Instagram':
        print(row)
        print('\n')
        

['Instagram', 'SOCIAL', '4.5', '66577313', 'Varies with device', '1,000,000,000+', 'Free', '0', 'Teen', 'Social', 'July 31, 2018', 'Varies with device', 'Varies with device']


['Instagram', 'SOCIAL', '4.5', '66577446', 'Varies with device', '1,000,000,000+', 'Free', '0', 'Teen', 'Social', 'July 31, 2018', 'Varies with device', 'Varies with device']


['Instagram', 'SOCIAL', '4.5', '66577313', 'Varies with device', '1,000,000,000+', 'Free', '0', 'Teen', 'Social', 'July 31, 2018', 'Varies with device', 'Varies with device']


['Instagram', 'SOCIAL', '4.5', '66509917', 'Varies with device', '1,000,000,000+', 'Free', '0', 'Teen', 'Social', 'July 31, 2018', 'Varies with device', 'Varies with device']




Some apps have multiple entries. Here with Instagram we see 4 different entries that only vary in the number of reviews. We intend to only keep the row with the most reviews since that indicates it is probably the most recent entry. This is the method we will use for all duplicates

In [11]:
"""
Creating a dictionary that has 
unique app names as the key and
the highest review count as the value
"""

max_reviews = {}
for row in android:
    app = row[0]
    reviews = int(row[3])
    if app not in max_reviews:
        max_reviews[app] = reviews
    elif (app in max_reviews) & (reviews > max_reviews[app]):
        max_reviews[app] = reviews
print('max reviews of Instagram:', max_reviews['Instagram'])
print('length of max_reviews:', len(max_reviews))


max reviews of Instagram: 66577446
length of max_reviews: 9659


In [12]:
android_clean = []
already_added = []
for row in android:
    name = row[0]
    reviews = int(row[3])
    if (reviews == max_reviews[name]) & (name not in already_added):
        android_clean.append(row)
        already_added.append(name)
        
explore_data(android_clean, 0, 2, True)
print('length of already_added:', len(already_added))


['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up']


['U Launcher Lite – FREE Live Cool Themes, Hide Apps', 'ART_AND_DESIGN', '4.7', '87510', '8.7M', '5,000,000+', 'Free', '0', 'Everyone', 'Art & Design', 'August 1, 2018', '1.2.4', '4.0.3 and up']


Number of rows: 9659
Number of columns: 13
length of already_added: 9659


#### Apple/ios

In [13]:
unique_apps_ios = []
for row in ios:
    name = row[1]
    if name not in unique_apps_ios:
        unique_apps_ios.append(name)
print('length of ios df:', len(ios))
print('length of unique_apps_ios:', len(unique_apps_ios))
        

length of ios df: 7197
length of unique_apps_ios: 7195


The IOS dataset does not contain any duplicates

### Remove non-english apps

In [14]:
"""
Function to determine if app name contains english characters
"""

def eng_ltr(string):
    counter = 0
    for char in string:
        if ord(char) > 127:
            counter += 1
        if counter > 3:         # allowing 3 non-english characters 
            return False
    return True
        
test_names = [
    
    'Instagram',
    '爱奇艺PPS -《欢乐颂2》电视剧热播',
    'Docs To Go™ Free Office Suite',
    'Instachat 😜'

]
for name in test_names:
    print(eng_ltr(name))
    

True
False
True
True


#### Google/Android

In [15]:
android_eng = []
for row in android_clean:
    if eng_ltr(row[0]):            # row[0] = app_name
        android_eng.append(row)
    

In [16]:
explore_data(android_eng, 0, 2, True)


['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up']


['U Launcher Lite – FREE Live Cool Themes, Hide Apps', 'ART_AND_DESIGN', '4.7', '87510', '8.7M', '5,000,000+', 'Free', '0', 'Everyone', 'Art & Design', 'August 1, 2018', '1.2.4', '4.0.3 and up']


Number of rows: 9614
Number of columns: 13


#### Apple/ios

In [17]:
ios_eng = []
for row in ios:
    if eng_ltr(row[1]):            # row[1] = app_name
        ios_eng.append(row)
        

In [18]:
explore_data(ios_eng, 0, 2, True)


['284882215', 'Facebook', '389879808', 'USD', '0.0', '2974676', '212', '3.5', '3.5', '95.0', '4+', 'Social Networking', '37', '1', '29', '1']


['389801252', 'Instagram', '113954816', 'USD', '0.0', '2161558', '1289', '4.5', '4.0', '10.23', '12+', 'Photo & Video', '37', '0', '29', '1']


Number of rows: 6183
Number of columns: 16


### Isolating free apps

#### Google/Android

In [19]:
prices = []
for row in android_eng:
    prices.append(row[7])            # row[7] = app_price

from pandas import value_counts
value_counts(prices).head()


0        8864
$0.99     145
$2.99     124
$1.99      73
$4.99      70
dtype: int64

In [20]:
android_data = []
for row in android_eng:
    if row[7] == '0':
        android_data.append(row)
        

In [21]:
explore_data(android_data, 0, 2, True)


['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up']


['U Launcher Lite – FREE Live Cool Themes, Hide Apps', 'ART_AND_DESIGN', '4.7', '87510', '8.7M', '5,000,000+', 'Free', '0', 'Everyone', 'Art & Design', 'August 1, 2018', '1.2.4', '4.0.3 and up']


Number of rows: 8864
Number of columns: 13


#### Apple/ios

In [22]:
prices = []
for row in ios_eng:
    prices.append(row[4])            # row[4] = app_price

from pandas import value_counts
value_counts(prices).head()


0.0     3222
2.99     669
0.99     641
1.99     610
4.99     375
dtype: int64

In [23]:
ios_data = []
for row in ios_eng:
    if row[4] == '0.0':
        ios_data.append(row)
        

In [24]:
explore_data(ios_data, 0, 2, True)


['284882215', 'Facebook', '389879808', 'USD', '0.0', '2974676', '212', '3.5', '3.5', '95.0', '4+', 'Social Networking', '37', '1', '29', '1']


['389801252', 'Instagram', '113954816', 'USD', '0.0', '2161558', '1289', '4.5', '4.0', '10.23', '12+', 'Photo & Video', '37', '0', '29', '1']


Number of rows: 3222
Number of columns: 16


# Most common apps by genre

In [25]:
## reprinting header and sample of data

print(android_header)
print(android_data[0])
print('\n')
print(ios_header)
print(ios_data[0])


['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type', 'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver', 'Android Ver']
['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up']


['id', 'track_name', 'size_bytes', 'currency', 'price', 'rating_count_tot', 'rating_count_ver', 'user_rating', 'user_rating_ver', 'ver', 'cont_rating', 'prime_genre', 'sup_devices.num', 'ipadSc_urls.num', 'lang.num', 'vpp_lic']
['284882215', 'Facebook', '389879808', 'USD', '0.0', '2974676', '212', '3.5', '3.5', '95.0', '4+', 'Social Networking', '37', '1', '29', '1']


In [26]:
"""

This function generates a frequency table,
given a dataset and an index number for the desired column

"""

def freq_table(dataset, index):
    ft = {}
    for row in dataset:
        param = row[index]
        if param in ft:
            ft[param] += 1
        else:
            ft[param] = 1
    for i in ft:
        ft[i] /= (len(dataset)*.01)
    return ft


In [27]:
"""

This function displays the top 10 values of a frequency table,
given a dataset and an index number for the desired column
it calls the freq_table function written above

"""

def display_table(dataset, index):
    table = freq_table(dataset, index)
    table_display = []
    for key in table:
        key_val_as_tuple = (table[key], key)
        table_display.append(key_val_as_tuple)

    table_sorted = sorted(table_display, reverse = True)
    for entry in table_sorted[:10]:           # only interested in the top values
        print(entry[1], ':', entry[0])
        

#### Google/Android

In [28]:
display_table(android_data, 9)    # row[9] = genres


Tools : 8.449909747292418
Entertainment : 6.069494584837545
Education : 5.347472924187725
Business : 4.591606498194946
Productivity : 3.892148014440433
Lifestyle : 3.892148014440433
Finance : 3.700361010830325
Medical : 3.5311371841155235
Sports : 3.463447653429603
Personalization : 3.3167870036101084


Tools and Entertainment genres are the most common genres here, but only with 8.4% and 6.1%. This doesn't indicate too much

In [29]:
display_table(android_data, 1)    # row[1] = categories


FAMILY : 18.907942238267147
GAME : 9.724729241877256
TOOLS : 8.461191335740072
BUSINESS : 4.591606498194946
LIFESTYLE : 3.9034296028880866
PRODUCTIVITY : 3.892148014440433
FINANCE : 3.700361010830325
MEDICAL : 3.5311371841155235
SPORTS : 3.395758122743682
PERSONALIZATION : 3.3167870036101084


FAMILY, GAME, AND TOOLS categories are the most common genres here, and FAMILY stands out with 18.9%. We have also now seen Tools pop up twice

#### Apple/ios

In [30]:
display_table(ios_data, 11)      # row[11] = prime_genre


Games : 58.16263190564867
Entertainment : 7.883302296710118
Photo & Video : 4.9658597144630665
Education : 3.6623215394165114
Social Networking : 3.289882060831782
Shopping : 2.60707635009311
Utilities : 2.5139664804469275
Sports : 2.1415270018621975
Music : 2.048417132216015
Health & Fitness : 2.017380509000621


In the Apple dataset, Games is by far the most common genre (58.2%), with Entertainment being a distant runner-up with 7.9%. This dataset seems to be dominated by apps designed for fun.

This Information indicates that the most common genre of app is a game. However, the Google Play store has a balance of practical and fun apps.

# Most popular apps by genre

#### Google/Android

In [31]:
display_table(android_data, 5)     # row[5] = number of app installs

1,000,000+ : 15.72653429602888
100,000+ : 11.552346570397113
10,000,000+ : 10.548285198555957
10,000+ : 10.1985559566787
1,000+ : 8.393501805054152
100+ : 6.915613718411552
5,000,000+ : 6.825361010830325
500,000+ : 5.561823104693141
50,000+ : 4.772111913357401
5,000+ : 4.512635379061372


In [32]:
"""

Here I will be calculating the average number of installs per app in a genre as a measure of popularity.

The number of installs row in the android dataset contains commas and plus signs,
in order to treat these strings as an integer I first remove these, and consider the
remaining integer a close aproximation of the actual value

"""

for row in android_data:
    row[5] = int(row[5].replace(',','').replace('+', ''))

category_ft = freq_table(android_data, 1)

category_popular = []
for category in category_ft:
    total = 0
    len_category = 0
    for row in android_data:
        if category == row[1]:
            total += row[5]            # row[5] = number of installs
            len_category += 1
    avg_install = round(total / len_category)
    category_popular.append([avg_install, category])
    category_popular = sorted(category_popular, reverse=True)
for i in category_popular[:10]:
    print(i, '\n')
    

[38456119, 'COMMUNICATION'] 

[24727872, 'VIDEO_PLAYERS'] 

[23253652, 'SOCIAL'] 

[17840110, 'PHOTOGRAPHY'] 

[16787331, 'PRODUCTIVITY'] 

[15588016, 'GAME'] 

[13984078, 'TRAVEL_AND_LOCAL'] 

[11640706, 'ENTERTAINMENT'] 

[10801391, 'TOOLS'] 

[9549178, 'NEWS_AND_MAGAZINES'] 



In [33]:
for row in android_data:
    if (row[1] == 'COMMUNICATION') & (row[5] > 100000000):
        print(row[0],row[5])

WhatsApp Messenger 1000000000
Google Duo - High Quality Video Calls 500000000
Messenger – Text and Video Chat for Free 1000000000
imo free video calls and chat 500000000
Skype - free IM & video calls 1000000000
LINE: Free Calls & Messages 500000000
Google Chrome: Fast & Secure 1000000000
UC Browser - Fast Download Private & Secure 500000000
Gmail 1000000000
Hangouts 1000000000
Viber Messenger 500000000


In [34]:
for row in android_data:
    if (row[1] == 'VIDEO_PLAYERS') & (row[5] > 100000000):
        print(row[0],row[5])

YouTube 1000000000
Google Play Movies & TV 1000000000
MX Player 500000000


In [35]:
for row in android_data:
    if (row[1] == 'SOCIAL') & (row[5] > 100000000):
        print(row[0],row[5])

Facebook 1000000000
Facebook Lite 500000000
Google+ 1000000000
Instagram 1000000000
Snapchat 500000000


Communication, Video Players, and Social apps dominate the Google Play store for popularity, however the top apps in these categories and dominated by big names like Facebook, YouTube, Gmail, WhatsApp. It might be a big leap to try and compete with these names. Lets look at the Apple data then try to come to a solution

#### Apple/ios

In [36]:
"""

The Apple dataset does not contain a total installs value
Therefore, I will be using the total_review_count, row[5]

"""

display_table(ios_data, 5)   # row[5] = total number of user ratings


0 : 4.686530105524519
1 : 0.7138423339540658
7 : 0.43451272501551835
5 : 0.43451272501551835
2 : 0.43451272501551835
10 : 0.40347610180012417
6 : 0.37243947858473
14 : 0.37243947858473
9 : 0.31036623215394166
53 : 0.31036623215394166


In [37]:
ios_genre_ft = freq_table(ios_data, 11)  # freq_table for genres

genre_popular = []
for genre in ios_genre_ft:
    total = 0
    len_genre = 0
    for row in ios_data:
        genre_app = row[11]
        if genre_app == genre:
            total += int(row[5])      # row[5] = total number of user ratings
            len_genre += 1
    avg_user_ratings = round(total / len_genre, 2)
    genre_popular.append([avg_user_ratings, genre])
genre_popular = sorted(genre_popular, reverse=True)
for i in genre_popular:
    print(i,'\n')


    

[86090.33, 'Navigation'] 

[74942.11, 'Reference'] 

[71548.35, 'Social Networking'] 

[57326.53, 'Music'] 

[52279.89, 'Weather'] 

[39758.5, 'Book'] 

[33333.92, 'Food & Drink'] 

[31467.94, 'Finance'] 

[28441.54, 'Photo & Video'] 

[28243.8, 'Travel'] 

[26919.69, 'Shopping'] 

[23298.02, 'Health & Fitness'] 

[23008.9, 'Sports'] 

[22788.67, 'Games'] 

[21248.02, 'News'] 

[21028.41, 'Productivity'] 

[18684.46, 'Utilities'] 

[16485.76, 'Lifestyle'] 

[14029.83, 'Entertainment'] 

[7491.12, 'Business'] 

[7003.98, 'Education'] 

[4004.0, 'Catalogs'] 

[612.0, 'Medical'] 



In [38]:
for row in ios_data:
    if (row[11] == 'Navigation') & (int(row[5]) > 100000):
        print(row[1],row[5])

Waze - GPS Navigation, Maps & Real-time Traffic 345046
Google Maps - Navigation & Transit 154911


In [39]:
for row in ios_data:
    if (row[11] == 'Reference') & (int(row[5]) > 1000):
        print(row[1],row[5])

Bible 985920
Dictionary.com Dictionary & Thesaurus 200047
Dictionary.com Dictionary & Thesaurus for iPad 54175
Google Translate 26786
Muslim Pro: Ramadan 2017 Prayer Times, Azan, Quran 18418
New Furniture Mods - Pocket Wiki & Game Tools for Minecraft PC Edition 17588
Merriam-Webster Dictionary 16849
Night Sky 12122
City Maps for Minecraft PE - The Best Maps for Minecraft Pocket Edition (MCPE) 8535
LUCKY BLOCK MOD ™ for Minecraft PC Edition - The Best Pocket Wiki & Mods Installer Tools 4693
GUNS MODS for Minecraft PC Edition - Mods Tools 1497


In [40]:
for row in ios_data:
    if (row[11] == 'Social Networking') & (int(row[5]) > 100000):
        print(row[1],row[5])

Facebook 2974676
Pinterest 1061624
Skype for iPhone 373519
Messenger 351466
Tumblr 334293
WhatsApp Messenger 287589
Kik 260965
ooVoo – Free Video Call, Text and Voice 177501
TextNow - Unlimited Text + Calls 164963
Viber Messenger – Text & Call 164249
Followers - Social Analytics For Instagram 112778


In the Apple store we see similar cases as with Google Play, Social Networking and Navigation genres are extremely popular, but only because they are dominated by giant companies. However, the Reference category is extremely popular and while there are some larger names in this category, the category doesnt seem to be dominated by the big names.

# My recommendation to the app developers is to develop an app in the reference category, religious reference apps and gaming reference apps in particular appear to be quite successful.